In [1]:
import random
import torch.nn as nn
import numpy as np
import torch
import math
import torch.optim as optim
from torch.utils.data import DataLoader

root = '../../'
bits = 16

In [2]:
# == LOAD IN THE DATA ==
X_train = torch.tensor( np.load( root + "Features/train_features_vgg16_cifar10.npy" ) )
y_train = np.load( root + "Features/train_labels_vgg16_cifar10.npy" )
y_train = torch.nn.functional.one_hot(torch.tensor(y_train), 10)

# == MAKE DATA LOADER ==
train_data = []
for i in range(len(X_train)):
   train_data.append([X_train[i], y_train[i]])
dataloader = DataLoader(train_data, batch_size=100, shuffle=True)

# == LOOP THROUGH THE BATCHES ==
for batch in dataloader:
    X_batch = batch[0]
    y_batch = batch[1]    

    break # ! REMOVE

In [3]:
model = nn.Sequential(  nn.Linear(4096,256),
                        nn.ReLU(),
                        nn.Linear(256, bits),
                        )

In [ ]:
class DTSHLoss(torch.nn.Module):
    def __init__(self):
        super(DTSHLoss, self).__init__()

    def forward(self, u, y):
        LAMBDA = 1
        ALPHA  = 1


        inner_product = u @ u.t()   # Similarity Matrix
        s = y @ y.t() > 0           # A matrix that show if the two idexes are the same or not
        count = 0

        loss1 = 0
        for row in range(s.shape[0]):
            # if has positive pairs and negative pairs
            if s[row].sum() != 0 and (~s[row]).sum() != 0:
                count += 1
                theta_positive = inner_product[row][s[row] == 1]
                theta_negative = inner_product[row][s[row] == 0]
                triple = (theta_positive.unsqueeze(1) - theta_negative.unsqueeze(0) - ALPHA ).clamp(min=-100,
                                                                                                             max=50)
                loss1 += -(triple - torch.log(1 + torch.exp(triple))).mean()

        if count != 0:
            loss1 = loss1 / count
        else:
            loss1 = 0

        loss2 = LAMBDA * (u - u.sign()).pow(2).mean()

        return loss1 + loss2

In [5]:
criterion = DTSHLoss()
optimizer = optim.RMSprop(model.parameters(), lr=1e-5 , weight_decay=1e-5)

In [6]:
for i in range(500):
    optimizer.zero_grad()

    u = model(X_batch)
    loss = criterion(u, y_batch.float())
    loss.backward()
    print(loss)
    optimizer.step()

tensor(1.8779, grad_fn=<AddBackward0>)
tensor(1.5266, grad_fn=<AddBackward0>)
tensor(1.2412, grad_fn=<AddBackward0>)
tensor(1.0312, grad_fn=<AddBackward0>)
tensor(0.9106, grad_fn=<AddBackward0>)
tensor(0.8380, grad_fn=<AddBackward0>)
tensor(0.7700, grad_fn=<AddBackward0>)
tensor(0.6947, grad_fn=<AddBackward0>)
tensor(0.6386, grad_fn=<AddBackward0>)
tensor(0.5957, grad_fn=<AddBackward0>)
tensor(0.5633, grad_fn=<AddBackward0>)
tensor(0.5358, grad_fn=<AddBackward0>)
tensor(0.5126, grad_fn=<AddBackward0>)
tensor(0.4934, grad_fn=<AddBackward0>)
tensor(0.4768, grad_fn=<AddBackward0>)
tensor(0.4604, grad_fn=<AddBackward0>)
tensor(0.4461, grad_fn=<AddBackward0>)
tensor(0.4317, grad_fn=<AddBackward0>)
tensor(0.4187, grad_fn=<AddBackward0>)
tensor(0.4069, grad_fn=<AddBackward0>)
tensor(0.3960, grad_fn=<AddBackward0>)
tensor(0.3859, grad_fn=<AddBackward0>)
tensor(0.3767, grad_fn=<AddBackward0>)
tensor(0.3684, grad_fn=<AddBackward0>)
tensor(0.3606, grad_fn=<AddBackward0>)
tensor(0.3535, grad_fn=<A

In [9]:
u

tensor([[ 0.8179, -0.9032,  1.0145,  ...,  1.0402,  0.9963,  0.9692],
        [-0.9606,  0.8932,  1.0683,  ...,  1.0014,  1.0060,  1.0243],
        [-0.5900, -0.7853,  0.3705,  ...,  1.0133,  0.8680,  1.0347],
        ...,
        [-1.2133,  0.8336,  0.9857,  ..., -1.0732, -0.9988,  0.9870],
        [-0.9323, -0.7978,  0.9990,  ...,  1.0519,  1.0628,  0.9720],
        [-0.8533, -0.9227,  0.9291,  ...,  1.0494,  1.0588,  0.9794]],
       grad_fn=<AddmmBackward0>)